## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-29-15-04-19 +0000,bbc,Assisted dying bill 'no hope' of passing unles...,https://www.bbc.com/news/articles/cr57j47811yo...
1,2026-01-29-15-00-16 +0000,nypost,Urgent Amber Alert issued for two abducted chi...,https://nypost.com/2026/01/29/us-news/amber-al...
2,2026-01-29-14-59-44 +0000,48hills,"BIG WEEK: Edwardian Ball, Pivot Festival, Miss...",https://48hills.org/2026/01/big-week-edwardian...
3,2026-01-29-14-59-00 +0000,wsj,Microsoft Shares Dive as Data-Center Spending ...,https://www.wsj.com/tech/ai/microsofts-earning...
4,2026-01-29-14-55-58 +0000,bbc,Heavy gunfire and blasts heard near airport in...,https://www.bbc.com/news/articles/cvgp47neze4o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2461/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
338,trump,47
155,ice,22
836,new,19
53,minnesota,19
85,minneapolis,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
290,2026-01-28-22-21-00 +0000,wsj,President Trump rose to power following a simp...,https://www.wsj.com/politics/policy/trump-and-...,130
201,2026-01-29-03-00-00 +0000,wsj,Fed Chair Jerome Powell’s decision on whether ...,https://www.wsj.com/economy/central-banking/je...,129
212,2026-01-29-02-07-32 +0000,nyt,The Trump administration moves to sell a landm...,https://www.nytimes.com/live/2026/01/28/us/tru...,121
173,2026-01-29-05-47-03 +0000,bbc,Trump warns Iran 'time is running out' for nuc...,https://www.bbc.com/news/articles/cly5pd459gko...,106
225,2026-01-29-01-25-01 +0000,nyt,Trump Administration Denies That Coercion Is a...,https://www.nytimes.com/2026/01/28/us/minnesot...,101


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
290,130,2026-01-28-22-21-00 +0000,wsj,President Trump rose to power following a simp...,https://www.wsj.com/politics/policy/trump-and-...
82,91,2026-01-29-12-00-01 +0000,startribune,AI image of Alex Pretti’s killing is the lates...,https://www.startribune.com/ai-image-of-alex-p...
193,71,2026-01-29-03-15-00 +0000,wsj,Google co-founder Sergey Brin has contributed ...,https://www.wsj.com/politics/policy/sergey-bri...
201,71,2026-01-29-03-00-00 +0000,wsj,Fed Chair Jerome Powell’s decision on whether ...,https://www.wsj.com/economy/central-banking/je...
47,62,2026-01-29-13-31-00 +0000,wsj,Democratic Sen. Amy Klobuchar of Minnesota ann...,https://www.wsj.com/politics/elections/sen-amy...
173,59,2026-01-29-05-47-03 +0000,bbc,Trump warns Iran 'time is running out' for nuc...,https://www.bbc.com/news/articles/cly5pd459gko...
128,48,2026-01-29-10-30-21 +0000,nypost,Texas cheerleader Grace Brito dies days after ...,https://nypost.com/2026/01/29/us-news/texas-ch...
258,39,2026-01-28-23-56-00 +0000,wsj,FBI Agents Search Election Office in Georgia’s...,https://www.wsj.com/politics/elections/fbi-age...
26,38,2026-01-29-14-20-59 +0000,nypost,Wild video shows wheel fall off passenger plan...,https://nypost.com/2026/01/29/us-news/wild-vid...
287,36,2026-01-28-22-35-00 +0000,wsj,The Federal Reserve entered a new holding patt...,https://www.wsj.com/economy/central-banking/fe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
